# クイックスタート3

このノートのクイックスタートは、Microsoft の Azure AI Foundry を使って、RAG を行う AI 推論チャット アプリケーションの開発を行うための学習ノートです。

※この内容は 2025年2月時点での仕様等に基づくものです。仕様やドキュメントの変更に伴い、この内容が古くなる可能性があります。

## 前提条件


このノートの内容を学習するにあたって、以下が必要になります。

- Visial Studio Code または このノートブックを実行可能な Jupyter が動作する環境
- Azure サブスクリプション
- 作成済みの Azure AI Foundry プロジェクトとデプロイしたモデル
- Azure AI Foundry プロジェクトに接続済みの Application Insights インスタンス（トレースに利用） 
- Python 3.8 以降
- Python に関する基礎知識
- 必要な Python パッケージ（下記のコードでインストールする）


In [ ]:
# 必要な Python パッケージのインストール
%pip install python-dotenv azure-identity azure-ai-projects azure-ai-inference azure-search-documents azure-core-tracing-opentelemetry azure-monitor-opentelemetry opentelemetry-sdk

## Azure AI Foundry で RAG を行う AI 推論チャットアプリを開発する


このノートでは、Azure AI Foundry SDK を使用して、独自のデータを使って RAG を行うシンプルな推論チャットアプリケーションを開発する方法について説明します。

Azure AI Search を使って架空のアウトドア製品のサンプルデータの検索インデックスを事前に登録しておきます。ユーザーからのクエリに対して、検索インデックスを使ってクエリに対する類似度の高いドキュメントを取り出し、取り出したドキュメントをグラウンディングデータとしてクエリと合わせて推論モデルに問い合わせを行って回答を得ます。

クエリに対する類似度の高いドキュメントの取り出しを行うために、ベクトル類似検索を行います。事前に検索インデックス登録するデータに対して、埋め込みモデルを使ってベクトル化を行い、ベクトル化したデータも合わせて検索インデックスに登録します。また、ユーザーからのクエリに対しても、同様に、ベクトル化を行って、Azure AI Search を使って検索して、ドキュメントを取り出します。


### 準備


このクイック スタートを始める前に、Azure AI Foundry ポータルから、ハブ リソースとプロジェクト リソースを作成して `gpt-4o-mini` モデルをプロジェクトへデプロイしてください。

また、検索インデックスの作成で利用する `text-embedding-ada-002` エンベディングモデルも合わせてデプロイしてください。

> [Azure AI Foundry プレイグラウンドのクイックスタート](https://learn.microsoft.com/ja-jp/azure/ai-studio/quickstarts/get-started-playground) を完了している場合、必要な作業は完了しています。

Visual Studio Code でこのノートを開き、[ドキュメントのこのセクション](https://code.visualstudio.com/docs/python/environments#_creating-environments) を参考にしながら、ワークスペースに Python 仮想環境を作成します。仮想環境を作成する際には、このフォルダにある `requirements.txt` を選択すると、このノートの Python アプリケーションコードの実行に必要なライブラリを Python 仮想環境にインストールすることができます。

仮想環境の準備ができたら、このノートにある Python コードをステップごとに実行します。


### チャットアプリのコードサンプル

#### 1. AI プロジェクトに接続します


サンプルデータをベースにした問い合わせを行う RAG チャットアプリのためのアプリの準備を行います。

  - `.env` ファイルの内容を環境変数に読み込みます。
  - 現在のユーザーで認証するための Azure 資格情報を取得します。
  - `AIProjectClient` の `from_connection_string()` を使って、Azure AI Foundry プロジェクトに接続します。


In [ ]:
import os
from dotenv import load_dotenv
from opentelemetry import trace
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

tracer = trace.get_tracer(__name__)

load_dotenv('.env', override=True)
project_connection_string = os.getenv('PROJECT_CONNECTION_STRING')
model_name_string = os.getenv('MODEL_NAME')
embeddings_model_name_string = os.getenv('EMBEDDINGS_MODEL_NAME')
search_index_name = os.environ.get("SEARCH_INDEX_NAME")

try:
    credential = DefaultAzureCredential()
    print(f"✅ Azure 資格情報の初期化が成功しました.")
except Exception as e:
    print(f"❌ Azure 資格情報の初期化が以下の理由で失敗しました: {str(e)}")

try:
    project = AIProjectClient.from_connection_string(
        conn_str=project_connection_string,
        credential=credential
    )
    print(f"✅ プロジェクトクライアントの初期化が成功しました.")
except Exception as e:
    print(f"❌ プロジェクトクライアントの初期化が以下の理由で失敗しました: {str(e)}")

#### 2. トレースを有効にします


以下のコードでは、チャットアプリケーションのトレースを有効にします。

> AI プロジェクトに Application Insights インスタンスを接続していない場合はこのセクションをスキップしてください。

- `AIInferenceInstrumentor` を使って、Azure AI 推論 SDK のインストルメンテーションを有効にします。
- `AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED` 環境変数を `true` に設定して、プロンプトと応答の内容をキャプチャします。
- Azure SDK トレースを有効にするには、`AZURE_SDK_TRACING_IMPLEMENTATION` 環境変数を `opentelemetry` に設定します。
  または、以下のコードを追加します。

    ```python
    from azure.core.settings import settings
    settings.tracing_implementation = "opentelemetry"
    ```
    
- トレース情報を Azure Application Insights (Azure Monitor) に送信するには、`configure_azure_monitor()` で有効にします。
    - ここでは Python 向けの `azure.monitor.opentelemetry` ライブラリを利用して、直接、送信（エクスポート）します。
    - AI プロジェクトに接続された Azure Application Insights への接続文字列を取得して、`configure_azure_monitor()`に渡します。


In [ ]:
from azure.ai.inference.tracing import AIInferenceInstrumentor
from azure.monitor.opentelemetry import configure_azure_monitor
from azure.core.settings import settings

# Enable Azure SDK tracing with either of two lines:
os.environ["AZURE_SDK_TRACING_IMPLEMENTATION"] = "opentelemetry"
settings.tracing_implementation = "opentelemetry"

# Enable tracing
AIInferenceInstrumentor().instrument()
# Enable logging message contents
os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true"

application_insights_connection_string = project.telemetry.get_connection_string()
if not application_insights_connection_string:
    print("❌ Application Insights がこのプロジェクトで有効になっていません.")
    print("==> Azure AI Foundry ポータルの \"トレース\"タブから、Application Inights を有効にしてください.")
else:
    configure_azure_monitor(connection_string=application_insights_connection_string)
    print("✅ Application Insights によるトレースを有効にしました.")


#### 3. 検索インデックスを定義します


以下のコードでは、検索インデックスのデータを入れるためのストア (`SearchIndex`) を定義して作成します。

取り扱うデータに合わせてフィールドを定義します。この例では、架空のアウトドア製品の情報を格納したファイル ([`outdoor_products_data.py`](./outdoor_products_data.py)) を使用します。
また、ベクトル検索で利用するベクトルデータを格納するフィールド (`contentVector`) も定義します。

検索インデックスの作成パラメーターは、[`search_index_config.py`](./search_index_config.py) で定義しています。ここでは詳しくは触れません。詳細については、Azure AI Search のドキュメントを参照ください ([1.セマンティックランク付け])  ([2.ベクトル検索によるランク付け])。

[1.セマンティックランク付け]:https://learn.microsoft.com/ja-jp/azure/search/semantic-search-overview

[2.ベクトル検索によるランク付け]:https://learn.microsoft.com/ja-jp/azure/search/semantic-search-overview

In [ ]:
from azure.search.documents.indexes.models import (
    SearchIndex, SearchField,
    SearchableField, SimpleField, SearchFieldDataType,
)
from search_index_config import semantic_search, vector_search

dimensions = 1536  # text-embedding-ada-002
if embeddings_model_name_string == "text-embedding-3-large":
    dimensions = 3072

fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="price", type=SearchFieldDataType.String),
    SearchableField(name="category", type=SearchFieldDataType.String),
    SearchableField(name="brand", type=SearchFieldDataType.String),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchField(
        name="contentVector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        # Size of the vector created by the text-embedding-ada-002 model.
        vector_search_dimensions=dimensions,
        vector_search_profile_name="myHnswProfile",
    ),
]

search_index_definitions = SearchIndex(
    name=search_index_name,
    fields=fields,
    semantic_search=semantic_search,
    vector_search=vector_search,
)

print(f"✅ 検索インデックスの定義が成功しました.")

#### 4. 検索インデックスに格納するドキュメント データを作成します

以下のコードでは、先ほど準備した検索インデックスに格納するドキュメント データ作成します。

AI 推論 API の埋め込みクライアントを使って、サンプルデータのアイテムごとに、コンテンツ フィールド (`content`) に対するベクトル検索データを作成 (`embed()`) します。

AI 推論 API の埋め込みモデルは、`.env` ファイルの `EMBEDDINGS_MODEL_NAME` で指定したモデルを使用します。このサンプルでは `text-embedding-ada-002` モデルも既定値として準備しています。AI プロジェクトには、このモデルを事前にデプロイしておきます。


In [ ]:
# アウトドア製品情報の読み込み
from outdoor_products_data import outdoor_products


@tracer.start_as_current_span("quick_start_03: create_search_index_docs")
def create_search_index_docs():
    try:
        embed_client = project.inference.get_embeddings_client()
        print("✅ 埋め込みクライアントを作成しました.")

        search_docs = []
        for doc in outdoor_products:
            embed_response = embed_client.embed(
                model=embeddings_model_name_string,
                input=doc["content"],
            )
            embed_vec = embed_response.data[0].embedding

            for item in embed_response.data:
                vec = item.embedding
                sample_str = f"[{vec[0]:.4f}, {vec[1]:.4f}, ..., {vec[-2]:.4f}, {vec[-1]:.4f}]"
                print(f"アイテム: '{doc["content"]}':\n" \
                      f"  埋め込みの長さ={len(vec)}\n" \
                      f"  ベクトルデータ: {sample_str}\n")

            search_docs.append(
              {
                  "id": doc["id"],
                  "title": doc["name"],
                  "price": str(doc["price"]) + "ドル",
                  "category": doc["category"],
                  "brand": doc["brand"],
                  "content": doc["content"],
                  "contentVector": embed_vec,
              }
            )

        print("✅ アウトドア製品のサンプルデータに対する検索インデックスデータが準備できました.")
        return search_docs

    except Exception as e:
        print("❌ 埋め込みクライアントの処理が以下の理由で失敗しました:", str(e))
        return []

search_index_documents = create_search_index_docs()

#### 5. 検索インデックスを作成し、ドキュメントをアップロードします

以下のコードを実行して、AI プロジェクトに接続された Azure AI Search の接続情報を取得します。


In [ ]:
from azure.ai.projects.models import ConnectionType
from azure.core.credentials import AzureKeyCredential


@tracer.start_as_current_span("quick_start_03: get_search_connection")
def get_search_connection():
    # Get the default search connection, with credentials
    search_connection = project.connections.get_default(
        connection_type=ConnectionType.AZURE_AI_SEARCH,
        include_credentials=True
    )
    if not search_connection:
        print("❌ 既定の Azure AI Search インスタンスへの接続が見つかりませんでした.")
    else:
        print("✅ 既定の Azure AI Search インスタンスへの接続を準備しました.")
    return search_connection

search_connection = get_search_connection()

以下のコードを実行して、`SearchIndexClient` クライアントを初期化して、定義した検索インデックスを Azure AI Search のインスタンス上に作成します。
`SEARCH_INDEX_NAME` 環境変数で定義された名前で、検索インデックスを作成します。

> すでに同じ名前の検索インデックスが存在する場合は、既存の検索インデックスを削除します。


In [ ]:
from azure.search.documents.indexes import SearchIndexClient


@tracer.start_as_current_span("quick_start_03: create_search_index")
def create_search_index(search_connection, search_index_definitions):
    try:
        index_client = SearchIndexClient(
            endpoint=search_connection.endpoint_url,
            credential=AzureKeyCredential(key=search_connection.key),
        )
        print("✅ SearchIndexClient を作成しました.")
        try:
            index_definition = index_client.get_index(search_index_name)
            index_client.delete_index(search_index_name)
            print(f"🗑️ 既存のインデックス ({search_index_name}) が見つかりましたので、削除しました.")
        # pylint: disable=broad-exception-caught
        except Exception:
            pass

        index_client.create_index(index=search_index_definitions)
        print(f"✅ インデックス {search_index_name}' を作成しました.")

    except Exception as e:
        print("❌ SearchIndexClient の作成が以下の理由で失敗しました:", str(e))

create_search_index(search_connection, search_index_definitions)

以下のコードを実行して、`SearchClient` クライアントを初期化して、作成した検索インデックスにドキュメント データをアップロードします。


In [ ]:
from azure.search.documents import SearchClient


@tracer.start_as_current_span("quick_start_03: upload_search_index_docs")
def upload_search_index_docs(search_connection, search_index_documents):
    try:
        search_client = SearchClient(
          endpoint=search_connection.endpoint_url,
          index_name=search_index_name,
          credential=AzureKeyCredential(search_connection.key)
        )
        print("✅ SearchClient クライアントを作成しました.")
        result = search_client.upload_documents(documents=search_index_documents)
        print(f"➕ {len(search_index_documents)} 個のドキュメントをインデックス {search_index_name}' にアップロードしました.")
    except Exception as e:
        print("❌ SearchClient クライアントの作成が以下の理由で失敗しました:", str(e))


upload_search_index_docs(search_connection, search_index_documents)

#### 6. RAG を使って登録したドキュメントでユーザーの質問へ回答します

以下のコードを実行して、RAG を行うチャット関数を定義します。

- ユーザーの質問に対して、埋め込みモデルでベクトルデータを作成します
- ユーザーの質問のベクトルデータに対してベクトル類似検索を使って、インデックス検索に登録したドキュメントデータに対して検索を行います
- 検索結果のドキュメントを含めたシステムメッセージを作成します
- システムメッセージとユーザーからの質問で、推論モデルに問い合わせを行います 

In [ ]:
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.search.documents.models import VectorizedQuery

@tracer.start_as_current_span("quick_start_03: rag_chat")
def rag_chat(query: str) -> str:
    try:
        # 1) ユーザーのクエリの埋め込みの取得
        embed_client = project.inference.get_embeddings_client()
        print("✅ 埋め込みクライアントを作成しました.")
        embedding = embed_client.embed(
            input=[query],
            model=embeddings_model_name_string,
        ).data[0].embedding

        # 2) ベクトル類似検索で検索を行う
        search_client = SearchClient(
          endpoint=search_connection.endpoint_url,
          index_name=search_index_name,
          credential=AzureKeyCredential(search_connection.key)
        )
        print("✅ SearchClient クライアントを作成しました.")
        vector_query = VectorizedQuery(vector=embedding, k_nearest_neighbors=50, fields="contentVector")
        results = search_client.search(
            search_text=None,
            vector_queries= [vector_query],
            select=["title", "category", "brand", "price", "content"],
            top=3,  # トップ3のドキュメントを取得します
        )
        top_docs_content = []
        for r in results:
            # Each doc is a SearchResult object with doc: dict.
            top_docs_content.append(
                f"""
                製品名: {r["title"]}
                カテゴリ：{r["category"]}
                ブランド名：{r["brand"]}
                価格：{r["price"]}
                製品の概要：
                {r["content"]}

                """
            )

        # 4) 取得したドキュメントを使ってシステムプロンプトを作成する
        # ここでは、ユーザーからのクエリに対する対応の手順を示したシステムメッセージを推論モデルに渡します
        system_text = (
            "# 手順\n"
            "- あなたは、アウトドア・キャンプ用品や衣類に関するクエリでユーザーを支援する AI アシスタントです。\n"
            "- 質問がアウトドア・キャンプ用品や衣類に関連しているが漠然としている場合は、"
            "ドキュメントを参照して回答するのではなく、明確にする質問をしてください。\n"
            "- 質問が一般的な場合 (たとえば、「それ」や「彼ら」が使用されている場合)、"
            "ユーザーに質問している製品を指定するように依頼してください。\n"
            "- 質問がアウトドア・キャンプ用品や衣類に関連している場合でも、該当する製品の情報がない場合は、その旨を伝えてください。\n"
            "- 質問がアウトドア・キャンプ用品や衣類に関連していない場合は、"
            "「申し訳ありませんが、アウトドア・キャンプ用品や衣類に関するクエリにしか回答できません。では、どのようにお手伝いすればよいでしょうか？」とだけ言ってください。\n"
            "- 答えをでっち上げないでください。\n"
            "- 以下のコンテキストを使用して、アウトドア/キャンプ用品と衣類に関する質問に、できるだけ完全に、正確に、簡潔に回答してください。\n"
            "\n"
            "# ドキュメント\n"
            "\n"
            + "\n".join(top_docs_content)
        )

        # 4) ドキュメントデータを含むシステムプロンプトでチャットを行う
        with project.inference.get_chat_completions_client() as rag_chat_client:
            response = rag_chat_client.complete(
                model=model_name_string,
                messages=[
                    SystemMessage(content=system_text),
                    UserMessage(content=query),
                ]
            )
        return response.choices[0].message.content

    except Exception as e:
        print("❌ RAG チャットの処理が以下の理由で失敗しました:", str(e))
        return None

#### 7. ユーザーの質問で RAG チャットを行います

先ほど作成した RAG チャット関数を、様々なユーザーの質問を使って呼び出し、結果を確認します。

In [ ]:
query = "TrekReady ハイキングブーツの価格はいくらですか？"

In [ ]:
query = "TrekReady ハイキングブーツの特徴を教えてください"

In [ ]:
query = "キャンプ用ストーブの価格はいくらですか？"

In [ ]:
query = "キャンプ用ストーブの中で一番のおすすめを教えてもらえますか？"

In [ ]:
query = "TrailMaster X4 テントの材質を教えてください。このテントは雨に強いですか？"

In [ ]:
query = "週末に6人でキャンプに行く予定です。新しいテントを探しています。おすすめを教えていただけますか?"

In [ ]:
query = "キャンプ用のバックパックを探しています。おすすめを教えていただけますか?"

In [ ]:
query = "日本の首都はどこですか?"

In [ ]:
if __name__ == "__main__":
    response = rag_chat(query=query)
    print(response)